In [1]:
from fastai.vision import *
path = Path('GASF')


In [2]:
def get_float_labels(y):
    label = ((y.parts[-1].split('_')[-1]).split('.')[0])
    return label

In [4]:
def get_subject_label(y):
    label = ((y.parts[-1].split('_')[-3]).split('.')[0])
    return label

In [14]:
def generate_data_withoutchild(n):
    blinded_file_list = []
    for fname in fnames_nomeds:
        if int(get_subject_label(fname)) != n:
            blinded_file_list.append(fname)
    return blinded_file_list

In [10]:
def collectpathsofsubject(n):
    subjectpaths = []
    for fname in fnames_nomeds:
        if get_subject_label(fname) == str(n):
            subjectpaths.append(fname)
    return subjectpaths
        

def givepredictionslistfromchildpaths(childpaths):
    predictionslist = []
    for path in childpaths:
        img = open_image(path)
        prediction = str(learn.predict(img)[0])
        predictionslist.append(prediction)
    return predictionslist

In [3]:
#generate new GASF list without on-ADHD GASFs

np.random.seed(42)
tfms = [[],[]]
fnames = get_image_files(path)
fnames_nomeds = []
for onefile in fnames:
    if "on-ADHD" not in onefile.parts[-1]:
        fnames_nomeds.append(onefile)

In [31]:
subjectlist_nomeds = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,\
                      25,26,27,28,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67]

In [27]:
subject_class_ratio_tuple_list = []

In [32]:
for n in subjectlist_nomeds:

    #removes child n GASFs from nomeds list
    blinded_file_list = generate_data_withoutchild(n)

    #creates a databunch without child n
    data = (ImageList(blinded_file_list)
            .split_by_rand_pct(0.2)
            .label_from_func(get_float_labels)
            .transform(tfms, size=224)
            .databunch())

    #trains a new model, naive to child n trials
    learn = cnn_learner(data, models.resnet50, metrics=error_rate)
    learn.fit_one_cycle(4)
    learn.fit_one_cycle(2, max_lr=2e-6)
    learn.save('intheblind_'+str(n))

    #reintroduces child n GASFs
    child_n_trial_paths = collectpathsofsubject(n)
    predictionslist_n = givepredictionslistfromchildpaths(childpaths=child_n_trial_paths)
    ratio_ADHD_to_all_preds = predictionslist_n.count('off-ADHD')/len(predictionslist_n)
    class_n = get_float_labels(child_n_trial_paths[0])
    subject_class_ratio_tuple = (n,class_n,ratio_ADHD_to_all_preds)


    subject_class_ratio_tuple

    with open('InTheBlind/'+str(n), 'w') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(subject_class_ratio_tuple)

    subject_class_ratio_tuple_list.append(subject_class_ratio_tuple)

with open('InTheBlind/full_list.txt', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(subject_class_ratio_tuple_list)

epoch,train_loss,valid_loss,error_rate,time
0,0.857448,0.755274,0.398084,01:27
1,0.676766,0.629451,0.344077,01:28
2,0.567337,0.637458,0.359756,01:28
3,0.478339,0.644816,0.351916,01:28


epoch,train_loss,valid_loss,error_rate,time
0,0.423881,0.647392,0.351916,01:28
1,0.425284,0.643890,0.347561,01:27


epoch,train_loss,valid_loss,error_rate,time
0,0.847425,0.745879,0.429688,01:29
1,0.670222,0.671495,0.387153,01:29
2,0.567732,0.675684,0.375868,01:30
3,0.469767,0.676614,0.363715,01:29


epoch,train_loss,valid_loss,error_rate,time
0,0.415365,0.674287,0.374132,01:29
1,0.413083,0.675021,0.376736,01:29


epoch,train_loss,valid_loss,error_rate,time
0,0.872526,0.713001,0.394281,01:32
1,0.682208,0.670962,0.389081,01:29
2,0.587446,0.679522,0.389081,01:29
3,0.477307,0.670794,0.380416,01:29


...

epoch,train_loss,valid_loss,error_rate,time
0,0.408232,0.671517,0.363874,01:28
1,0.407399,0.671822,0.365620,01:28


epoch,train_loss,valid_loss,error_rate,time
0,0.859304,0.721990,0.388502,01:28
1,0.660472,0.688774,0.367596,01:28
2,0.558073,0.687020,0.378920,01:28
3,0.468061,0.684692,0.367596,01:28


epoch,train_loss,valid_loss,error_rate,time
0,0.416230,0.684358,0.364983,01:28
1,0.407393,0.683380,0.368467,01:28


epoch,train_loss,valid_loss,error_rate,time
0,0.855720,0.771144,0.404534,01:28
1,0.677022,0.669627,0.360942,01:29
2,0.574193,0.662448,0.350480,01:28
3,0.475137,0.663719,0.349608,01:28


epoch,train_loss,valid_loss,error_rate,time
0,0.421268,0.664849,0.354839,01:28
1,0.428652,0.664130,0.349608,01:28
